In [41]:
#module dependencies
import os
import codecs
import pandas as pd
import csv
import vincent
import locale
from vincent import *
from IPython.core.display import HTML
vincent.core.initialize_notebook()

In [42]:
"""
Creates a pandas dataFrame from a state data file and converts strings to ints.
filename must be a csv file.
"""
def create_dataframe():
    filename = "/state_population_data_2007_to_2015.txt"
    column_names = ['state','2008','2009','2010','2011','2012','2013','2014','2015']
    pandas_dataframe = parse_data(filename,column_names)

    ## covert all the populations of form 4,123,456 to 4123456
    pandas_dataframe['2008'] = list(map(lambda number : int(number.replace(',', '')),pandas_dataframe['2008']))
    pandas_dataframe['2009'] = list(map(lambda number : int(number.replace(',', '')),pandas_dataframe['2009']))
    pandas_dataframe['2010'] = list(map(lambda number : int(number.replace(',', '')),pandas_dataframe['2010']))
    pandas_dataframe['2011'] = list(map(lambda number : int(number.replace(',', '')),pandas_dataframe['2011']))
    pandas_dataframe['2012'] = list(map(lambda number : int(number.replace(',', '')),pandas_dataframe['2012']))
    pandas_dataframe['2013'] = list(map(lambda number : int(number.replace(',', '')),pandas_dataframe['2013']))
    pandas_dataframe['2014'] = list(map(lambda number : int(number.replace(',', '')),pandas_dataframe['2014']))
    pandas_dataframe['2015'] = list(map(lambda number : int(number.replace(',', '')),pandas_dataframe['2015']))
    return pandas_dataframe


In [43]:
"""
Creates a raw pandas dataFrame from a data file.
file must be a csv file.
"""
def parse_data(filename, column_names):
    infile = codecs.open(os.getcwd() + filename, encoding='UTF-8')
    data = list(csv.reader(infile,delimiter = ','))
    pandas_dataframe = pd.DataFrame(data, columns = column_names)
    return pandas_dataframe

In [93]:
"""
Creates a vincent map of the United States 
"""
def create_vincent_us_state_map(year,pandas_dataframe):
    geo_data = [{'name': 'states',
                 'url': "https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_states.topo.json",
                 'feature': 'us_states.geo'}]

    vis = vincent.Map(data = pandas_dataframe, geo_data = geo_data, scale = 800,
                      projection = 'albersUsa', data_bind = year, data_key ='state',
                      map_key = {'states': 'properties.NAME'})
    #Custom threshold scale
    vis.scales[0].type='threshold'
    vis.marks[0].properties.enter.stroke_opacity = ValueRef(value = 0.5)
    vis.scales[0].domain = [600000,100000,3000000,6000000,15000000,20000000,30000000]
    vis.legend(title='Total populations for '+year)
    vis.to_json('vega.json')
    return vis

In [94]:
pandas_dataframe = create_dataframe()
vincent_map = create_vincent_us_state_map('2009',pandas_dataframe)
vincent_map.display()

In [114]:
pandas_dataframe['raw_growth_2014'] = pandas_dataframe['2015'] - pandas_dataframe['2013']
pandas_dataframe['population_ajusted_growth_2014'] = (pandas_dataframe['2015'] - pandas_dataframe['2013']) / (pandas_dataframe['2013']/1000)
#print(pandas_dataframe['growth_2009'])

In [118]:
vincent_map.rebind(column='raw_growth_2014', brew='YlGnBu')
# vincent_map.legend(title='Raw growth for 2009')
vincent_map.scales[0].type='threshold'
vincent_map.scales[0].domain = [-20000,1000, 5000,50000,100000, 400000]
vincent_map.to_json('vega.json')
vincent_map.display()


In [121]:
vincent_map.rebind(column='population_ajusted_growth_2014', brew='YlGnBu')
vincent_map.scales[0].type='threshold'
vincent_map.scales[0].domain = [-4,-2,0,2,5,10,20,30]
vincent_map.to_json('vega.json')
vincent_map.display()